In [1]:
#import libraries
import pandas as pd

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

a = pd.read_csv('Answers.csv', usecols = ['Id', 'OwnerUserId', 'CreationDate', 'ParentId', 'Score', 'Body'], low_memory=False)
q = pd.read_csv('Questions.csv', usecols = ['Id', 'OwnerUserId', 'CreationDate', 'ClosedDate', 'Score', 'Title', 'Body'], low_memory=False)
t = pd.read_csv('Tags.csv', usecols = ['Id', 'Tag'], low_memory=False)

q_clean = q
q_clean = q_clean[q_clean['Id'].str.contains('\D') == False]
q_clean['Id'] = q_clean['Id'].astype(str).astype(int)
q_clean = q_clean[q_clean['Score'].str.contains('\D') == False]
q_clean['Score'] = q_clean['Score'].astype(str).astype(int)
q_clean.drop(["OwnerUserId"], axis=1, inplace=True)
print(q_clean.info())

a_clean = a
a_clean.drop(["Id", "OwnerUserId"], axis=1, inplace=True)
a_clean.rename(columns={'ParentId' : 'Id'}, inplace=True)

<class 'pandas.core.frame.DataFrame'>
Index: 976367 entries, 0 to 1048574
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Id            976367 non-null  int32 
 1   CreationDate  976367 non-null  object
 2   ClosedDate    31217 non-null   object
 3   Score         976367 non-null  int32 
 4   Title         976367 non-null  object
 5   Body          976366 non-null  object
dtypes: int32(2), object(4)
memory usage: 44.7+ MB
None


In [18]:
print(t.shape) #1,048,575 rows
print(t['Tag'].unique().shape) # 21,984 unique values
#
unique_tags = t['Tag'].value_counts(ascending=False).reset_index()
# tags = pd.DataFrame(t[t.Tag == '*c*'])
# print(tags.shape)
# tags.head(15)

(1048575, 2)
(21984,)


<h1>Get all Id from posts with relevent tags</h1>

attempt 1:
only gets id's from posts tagged with the language. no information on libraries. 

In [ ]:
t_java = pd.DataFrame(t[t.Tag == 'java'])
t_python = pd.DataFrame(t[t.Tag == 'python'])
t_cpp = pd.DataFrame(t[t.Tag == 'c++'])
t_java = pd.DataFrame(t[t.Tag == 'java'])
t_python = pd.DataFrame(t[t.Tag == 'python'])
t_cpp = pd.DataFrame(t[t.Tag == 'c++'])

attempt 2:
gets id's from posts tagged with language or libraries. case insensitive, gets more cases where the language is hidden inside a larger string. slow and long imnplementation. doesn't separate by language. 

In [56]:
out1 = t[t['Tag'].astype(str).str.contains('java', case=False)]
out2 = t[t['Tag'].astype(str).str.contains('python', case=False)]
out3 = t[t['Tag'].astype(str).str.contains('c\+\+', case=False)]
out4 = t[t['Tag'].astype(str).str.contains('cpp', case=False)]
#t_cpp = pd.DataFrame(t[t.Tag == 'c++'])
frames = [out1, out2, out3, out4]
out = pd.concat(frames)
out.tail()

,Id,Tag
902207,11259570,cppcheck
962338,11916290,rcpp
1017367,12528930,activemq-cpp
1032255,12691710,rcpp
1047083,12857860,rcpp


attempt 3: speed and clean up implementation. separate by language

In [65]:
cpp_tags_list = ['c\+\+', 'cpp', 'boost', 'poco', 'googletest', 'tbb', 'cinder', 'wtl',
                 'stdvector', 'stdstring', 'stdlist']
c_tags_cpp_compatible = ['ffmpeg', 'openssl', 'sqlite', 'opencv']
java_tags_list = ['java', 'guava', 'cglib', 'apache', 'jackson', 'c3p0', 'jms',
                 'jodatime', 'gson', 'log4j', 'hslf', 'slf4j', 'junit', 
                 'mockito', 'spring', 'jmx', 'jaxb', 'httpclient'] #remember to remove Javascript!! 
python_tags_list = ['python', 'django', 'pybrain', 'keras', 'lightgbm', 'eli5', 'theano', 
                    'scipy', 'scrapy', 'scikit-learn', 'scikit', 'pygame', 'matplotlib', 'pytorch',
                    'pandas', 'numpy']
#libraries that encompass multiple languages
multiple_tags_list = ['eclipse', 'wpf', 'tensorflow', 'maven', 'xerces'] 
# eclipse is java and c, sometimes c++
# tensorflow is python and c++ and javascript
# maven is java and c++
# xerces is java and c++
#std can appear in many languages????
# WPF is C and C++


cpp_pattern = '|'.join(cpp_tags_list)
java_pattern = '|'.join(java_tags_list)
python_pattern = '|'.join(python_tags_list)

cpp_ids = t[t['Tag'].astype(str).str.contains(cpp_pattern, case=False)]
java_ids = t[t['Tag'].astype(str).str.contains(java_pattern, case=False)]
python_ids = t[t['Tag'].astype(str).str.contains(python_pattern, case=False)]

#remove .h from c++, remove petapoco
#remove Javascript, remove json, remove log4js




                 Tag  count
0                c++  14691
1         visual-c++   1012
2              boost    731
3              c++11    460
4            c++-cli    230
..               ...    ...
110            c++1z      1
111         yaml-cpp      1
112     c++-concepts      1
113  c++builder-2009      1
114       boost-move      1

[115 rows x 2 columns]
               Tag  count
0             java  29266
1       javascript  26199
2           spring   1954
3           apache   1770
4          java-ee    979
..             ...    ...
245    openid4java      1
246        javapos      1
247    java-canvas      1
248  spring-nature      1
249      spring-js      1

[250 rows x 2 columns]
                     Tag  count
0                 python  13470
1                 django   3862
2          django-models    522
3                  numpy    343
4             python-3.x    315
..                   ...    ...
194    ironpython-studio      1
195    python-webbrowser      1
196  django-ajax

In [66]:

print(cpp_ids['Tag'].value_counts(ascending=False).reset_index().head(10))
print(java_ids['Tag'].value_counts(ascending=False).reset_index().head(10))
print(python_ids['Tag'].value_counts(ascending=False).reset_index().head(10))

             Tag  count
0            c++  14691
1     visual-c++   1012
2          boost    731
3          c++11    460
4        c++-cli    230
5     boost-asio    106
6     c++builder     91
7           poco     78
8  objective-c++     50
9   boost-thread     38
                 Tag  count
0               java  29266
1         javascript  26199
2             spring   1954
3             apache   1770
4            java-ee    979
5         spring-mvc    690
6  javascript-events    647
7            java-me    473
8              junit    385
9    spring-security    270
                Tag  count
0            python  13470
1            django   3862
2     django-models    522
3             numpy    343
4        python-3.x    315
5  django-templates    302
6      django-admin    266
7        matplotlib    252
8      django-forms    249
9        python-2.7    222
